In [1]:
import pandas as pd, numpy as np
from rolling_framework import Machine

import warnings
warnings.filterwarnings('ignore')

/Users/ethan_hong/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Loading

In [2]:
period = '19712023'

#%% Target variables : excess returns
y = pd.read_csv(f'data/{period}/exrets_{period}.csv', index_col='Time')             # Target : Excess Return (Bond Risk Premia)
y = y[['xr_2','xr_5','xr_7','xr_10']]

#%% Predictors (Features)
FWDS = pd.read_csv(f'data/{period}/fwds_{period}.csv', index_col='Time')            # Forward Rates
MACV = pd.read_csv(f'data/{period}/MacroFactors_{period}.csv', index_col='Time')    # Ludvigson and Ng (2009), Macrovariables
LSC = pd.read_csv(f'data/{period}/lsc_{period}.csv', index_col='Time')              # Level, Slope and Curvature
# YLV = pd.read_csv(f'data/{period}/yl_{period}.csv', index_col='Time')               # yield-level
RVOL = pd.read_csv(f'data/{period}/real_vol_{period}.csv', index_col='Time')        # Realised Volatility 10yr
#IVOL = pd.read_csv(f'data/{period}/imp_vol_{period}.csv', index_col='Time')        # Implied Volatility (Work for 1990~)
CP = pd.read_csv(f'data/{period}/cp_{period}.csv', index_col='Time')                # Cochrane-Piazessi Factor
# YALL = pd.read_csv(f'data/{period}/yl_all_{period}.csv', index_col='Time')          # 12 m, 24 m, ... 120 m
# YMAX = pd.read_csv(f'data/{period}/yl_max_{period}.csv', index_col='Time')          # 1m ... 120 m

F6 = MACV[['F1','F2','F3','F4','F8','F1^3']]
SL = LSC[['slope']]
LV = LSC[['level']]
CU = LSC[['curvature']]


AGG = pd.concat([CP, FWDS], axis=1)

In [12]:
# ────────────────────────────
# Random-Forest
# ────────────────────────────
param_grid_rf = {
    "model__estimator__n_estimators":      [300],
    "model__estimator__max_depth":         [2, 8],
    "model__estimator__min_samples_split": [2, 4],
    "model__estimator__min_samples_leaf":  [1, 2, 4],
    "model__estimator__max_features":      [0.25, 0.5,1],
}

# ────────────────────────────
# Extra-Trees
# ────────────────────────────
param_grid_et = {
    "model__estimator__n_estimators":      [300],
    "model__estimator__max_depth":         [2, 8],
    "model__estimator__min_samples_split": [2, 4],
    "model__estimator__min_samples_leaf":  [1, 2, 4],
    "model__estimator__max_features":      [0.25, 0.5, 1],
}

# ────────────────────────────
# XGBoost  (XGBRegressor 사용 가정)
# ────────────────────────────
param_grid_xgb = {
    "model__estimator__n_estimators":  [300],
    "model__estimator__max_depth":     [2, 4],
    "model__estimator__learning_rate": [0.01],
    "model__estimator__subsample":     [0.7, 0.5],
    "model__estimator__reg_lambda":    [0.1, 1.0],
}

param_grid_lasso = {'reg__alpha': [0.001, 1.0]}    
param_grid_ridge      = {'reg__alpha': [0.001, 1]}
param_grid_elasticnet = {
    'reg__alpha':   [0.01, 0.1, 1, 10],
    'reg__l1_ratio':[0.1, 0.3, 0.5],
}

### Helper

In [4]:
# ────────────────────────────
# Custom Weights for Portfolio
# ────────────────────────────
w_custom = pd.Series(
    {
        "xr_2" : 2,   # 2Y 듀레이션
        "xr_5" : 5,   # 5Y 듀레이션
        "xr_7" : 7,   # 7Y 듀레이션
        "xr_10": 10,   # 10Y 듀레이션
    },
    name="w"
)

cols_dw = w_custom.index.tolist()
# 합이 1이 되도록 정규화
w_custom = w_custom / w_custom.sum()


# ────────────────────────────
bs = "199009" # burn-in-start
be = "200609" # burn-in-end
p = ["199009", "202312"] #period

### OLS, LASSO, ELASTICNET

In [5]:
ols_1 = Machine(SL, y, 'OLS' , burn_in_start=bs, burn_in_end=be, period=p)
ols_1.training()
ols_2 = Machine(CP,y, 'OLS', burn_in_start=bs, burn_in_end=be, period=p)
ols_2.training()
ols_3 = Machine(F6, y, 'OLS', burn_in_start=bs, burn_in_end=be, period=p)
ols_3.training()

print(ols_1.R2OOS())
print('ew :', ols_1.r2_oos_portfolio())
print('dw :', ols_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ols_2.R2OOS())
print('ew :', ols_2.r2_oos_portfolio())
print('dw :', ols_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ols_3.R2OOS())
print('ew :', ols_3.r2_oos_portfolio())
print('dw :', ols_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

OLS rolling: 100%|██████████| 208/208 [00:08<00:00, 23.55it/s]


xr_2    -0.112185
xr_5    -0.004982
xr_7     0.035567
xr_10    0.096039
dtype: float64
ew : 0.04726677148779712
dw : 0.06475501531237549
xr_2    -0.089217
xr_5    -0.023229
xr_7     0.009279
xr_10    0.068695
dtype: float64
ew : 0.027363835050630003
dw : 0.0415248375379742
xr_2    -0.909016
xr_5    -0.762650
xr_7    -0.730646
xr_10   -0.799235
dtype: float64
ew : -0.7910499151881054
dw : -0.7819287018906296


In [11]:
ridge_1 = Machine(SL, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_1.training()
ridge_2 = Machine(CP, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_2.training()
ridge_3 = Machine(F6, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_3.training()


print(ridge_1.R2OOS())
print('ew :', ridge_1.r2_oos_portfolio())
print('dw :', ridge_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ridge_2.R2OOS())
print('ew :', ridge_2.r2_oos_portfolio())
print('dw :', ridge_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ridge_3.R2OOS())
print('ew :', ridge_3.r2_oos_portfolio())
print('dw :', ridge_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 208/208 [00:03<00:00, 55.46it/s]


xr_2    -0.112158
xr_5    -0.005105
xr_7     0.035384
xr_10    0.095742
dtype: float64
ew : 0.04705223225758537
dw : 0.06451414410137912
xr_2    -0.089218
xr_5    -0.023174
xr_7     0.009335
xr_10    0.068707
dtype: float64
ew : 0.02739429299065743
dw : 0.04155276878841563
xr_2    -0.945394
xr_5    -0.785102
xr_7    -0.743835
xr_10   -0.800098
dtype: float64
ew : -0.8045260597437767
dw : -0.7911905796168832


In [6]:
lasso_1 = Machine(SL, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_1.training()
lasso_2 = Machine(CP, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_2.training()
lasso_3 = Machine(F6, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_3.training()


print(lasso_1.R2OOS())
print('ew :', lasso_1.r2_oos_portfolio())
print('dw :', lasso_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(lasso_2.R2OOS())
print('ew :', lasso_2.r2_oos_portfolio())
print('dw :', lasso_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(lasso_3.R2OOS())
print('ew :', lasso_3.r2_oos_portfolio())
print('dw :', lasso_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 208/208 [00:10<00:00, 20.40it/s]


xr_2    -0.107313
xr_5    -0.078389
xr_7    -0.048870
xr_10    0.039690
dtype: float64
ew : -0.020440336842246776
dw : -0.002134152092058139
xr_2    -0.206568
xr_5    -0.139219
xr_7    -0.060293
xr_10    0.026807
dtype: float64
ew : -0.042742389753450905
dw : -0.01893418154346027
xr_2    -0.100748
xr_5    -0.068164
xr_7    -0.061414
xr_10   -0.010356
dtype: float64
ew : -0.0423209549000958
dw : -0.03143878483906781


In [7]:
elasticnet_1 = Machine(SL, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_1.training()

elasticnet_2 = Machine(CP, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_2.training()

elasticnet_3 = Machine(F6, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_3.training()


print(elasticnet_1.R2OOS())
print('ew :', elasticnet_1.r2_oos_portfolio())
print('dw :', elasticnet_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(elasticnet_2.R2OOS())
print('ew :', elasticnet_2.r2_oos_portfolio())
print('dw :', elasticnet_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(elasticnet_3.R2OOS())
print('ew :', elasticnet_3.r2_oos_portfolio())
print('dw :', elasticnet_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 360/360 [01:01<00:00,  5.83it/s]


xr_2    -0.175665
xr_5    -0.089630
xr_7    -0.028150
xr_10    0.063809
dtype: float64
ew : -0.1658429596938562
dw : -0.14602176236112752
xr_2    -0.042565
xr_5     0.024434
xr_7     0.054527
xr_10    0.099454
dtype: float64
ew : -0.06800728526477418
dw : -0.0670683359962545
xr_2    -0.234712
xr_5    -0.089215
xr_7    -0.054463
xr_10   -0.005559
dtype: float64
ew : -0.20162131707054187
dw : -0.19458831277632793


### Tree

In [8]:
rf_1 = Machine(SL, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_1.training()
rf_2 = Machine(CP, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_2.training()
rf_3 = Machine(F6, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_3.training()

print(rf_1.R2OOS())
print('ew :', rf_1.r2_oos_portfolio())
print('dw :', rf_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(rf_2.R2OOS())
print('ew :', rf_2.r2_oos_portfolio())
print('dw :', rf_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(rf_3.R2OOS())
print('ew :', rf_3.r2_oos_portfolio())
print('dw :', rf_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Tree rolling: 100%|██████████| 208/208 [1:01:17<00:00, 17.68s/it]


xr_2    -0.721309
xr_5    -0.233886
xr_7    -0.126278
xr_10   -0.012651
dtype: float64
ew : -0.12682594400060188
dw : -0.08109252091980879
xr_2    -0.169770
xr_5    -0.095587
xr_7    -0.088162
xr_10   -0.025225
dtype: float64
ew : -0.06080850593126108
dw : -0.05180146270678243
xr_2    -0.271849
xr_5    -0.080663
xr_7    -0.053206
xr_10   -0.029005
dtype: float64
ew : -0.050005305156443525
dw : -0.038609828390823386


In [9]:
et_1 = Machine(SL, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_1.training()
et_2 = Machine(CP, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_2.training()
et_3 = Machine(F6, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_3.training()


print(et_1.R2OOS())
print('ew :', et_1.r2_oos_portfolio()) 
print('dw :', et_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(et_2.R2OOS())
print('ew :', et_2.r2_oos_portfolio())
print('dw :', et_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(et_3.R2OOS())
print('ew :', et_3.r2_oos_portfolio())
print('dw :', et_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


Tree rolling: 100%|██████████| 208/208 [29:53<00:00,  8.62s/it]


xr_2    -0.232331
xr_5    -0.044681
xr_7     0.006003
xr_10    0.064578
dtype: float64
ew : 0.0074013929188290195
dw : 0.030348309683687624
xr_2    -0.051354
xr_5    -0.019894
xr_7    -0.013372
xr_10    0.011711
dtype: float64
ew : -0.003257708171678031
dw : 0.0020672315607360714
xr_2    -0.258760
xr_5    -0.079659
xr_7    -0.053671
xr_10   -0.025667
dtype: float64
ew : -0.05128120892510046
dw : -0.038863526145635596


In [13]:
xgb_1 = Machine(SL, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_1.training()
xgb_2 = Machine(CP, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_2.training()
xgb_3 = Machine(F6, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_3.training()

print(xgb_1.R2OOS())
print('ew :', xgb_1.r2_oos_portfolio())
print('dw :', xgb_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(xgb_2.R2OOS())
print('ew :', xgb_2.r2_oos_portfolio())
print('dw :', xgb_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(xgb_3.R2OOS())
print('ew :', xgb_3.r2_oos_portfolio())
print('dw :', xgb_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Tree rolling: 100%|██████████| 208/208 [02:16<00:00,  1.52it/s]


xr_2    -0.679394
xr_5    -0.241951
xr_7    -0.111564
xr_10    0.006505
dtype: float64
ew : -0.11712575607778808
dw : -0.06710734799803997
xr_2    -0.058526
xr_5    -0.045104
xr_7    -0.055717
xr_10   -0.029156
dtype: float64
ew : -0.03902176223107512
dw : -0.03676165924995667
xr_2    -0.407986
xr_5    -0.168785
xr_7    -0.118883
xr_10   -0.064790
dtype: float64
ew : -0.11435786299241091
dw : -0.09344122922317832


### DNN : FWDs ONly

In [ ]:
param_grid = {
    "dnn__module__hidden": [(3,)],    
    "dnn__module__dropout": [0.2],
    "dnn__lr": [0.01],
    "dnn__optimizer__weight_decay": [0.0005],
}

dnn_1 = Machine(
    FWDS, y,
    model_type="DNN",                  
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12,
    random_state=15
)
dnn_1.training()
print(dnn_1.R2OOS())
print('ew :', dnn_1.r2_oos_portfolio())
print('dw :', dnn_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


### DNN : FWDS + MACRO

In [ ]:
param_grid = {
    "dnn__module__hidden": [(3,3)],    
    "dnn__module__dropout": [0.2],
    "dnn__lr": [0.01],
    "dnn__optimizer__weight_decay": [0.0005],
}

dnn_2 = Machine(
    FWDS, y,
    model_type="DNN",                  
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12,
    random_state=15
)
dnn_2.training()
print(dnn_2.R2OOS())
print('ew :', dnn_2.r2_oos_portfolio())
print('dw :', dnn_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


In [ ]:
AGG = pd.concat([MACV, CP], axis=1)

# ── 예시: dual-branch MLP 하이퍼파라미터 그리드 ─────────────────────────
param_grid = {
    # ── Branch-1 Macro
    "dnn__module__hidden1": [(32,)],    # 얕은 vs. 두 층
    "dnn__module__drop1":   [0.2],       # 드롭아웃 비율

    # ── Branch-2 FWDS : CP
    "dnn__module__hidden2": [(1)],
    "dnn__module__drop2":   [0.2],

    # ── 공통 optimizer & regularization ────────────────────────────────
    "dnn__optimizer__lr":           [0.001],
    "dnn__optimizer__weight_decay": [0.0005]
}

grp1_cols = [c for c in AGG.columns if c.startswith("F")]
grp2_cols = [c for c in AGG.columns if c not in grp1_cols]

dnn_d1 = Machine(
    AGG, y,
    model_type="DNN_DUAL",
    option={"grp1": grp1_cols, "grp2": grp2_cols},
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12
)
dnn_d1.training()
print(dnn_d1.R2OOS())

In [ ]:
AGG2 = pd.concat([MACV, FWDS], axis=1)

# ── 예시: dual-branch MLP 하이퍼파라미터 그리드 ─────────────────────────
param_grid = {
    # ── Branch-1 Macro
    "dnn__module__hidden1": [(32,)],    # 얕은 vs. 두 층
    "dnn__module__drop1":   [0.2],       # 드롭아웃 비율

    # ── Branch-2 FWDS : CP
    "dnn__module__hidden2": [(3,3)],
    "dnn__module__drop2":   [0.2],

    # ── 공통 optimizer & regularization ────────────────────────────────
    "dnn__optimizer__lr":           [0.001],
    "dnn__optimizer__weight_decay": [0.0005]
}

grp1_cols = [c for c in AGG2.columns if c.startswith("F")]
grp2_cols = [c for c in AGG2.columns if c not in grp1_cols]

dnn_d2 = Machine(
    AGG2, y,
    model_type="DNN_DUAL",
    option={"grp1": grp1_cols, "grp2": grp2_cols},
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    params_grid=param_grid,
    forecast_horizon=12
)
dnn_d2.training()
print(dnn_d2.R2OOS())